In [1]:
from __future__ import print_function, division

# BERT
import torch
from torch.nn import CosineSimilarity
import numpy as np

# XLNet
import transformers

# Paper
import json
import random

import debiaswe as dwe
import debiaswe.we as we
from debiaswe.we import WordEmbedding
from debiaswe.data import load_professions

## Similarity function

In [2]:
def similarity(words, encodings, word_idx, measure="cos"):
    cos = CosineSimilarity(dim=0)
    similarities = []
    sims = []
    print("Similarity with \'"+words[word_idx]+"\'")
    for i in range(len(words)):
        # cos or torch.dist
        sim = None
        if measure == "cos":
            sim = cos(encodings[i], encodings[word_idx])
        elif measure == "euclidean":
            sim = torch.dist(encodings[i], encodings[word_idx])
        else:
            print("Unknown distance metric")
        sims.append((words[i], sim.item()))
#         print(words[i]+"\t"+str(sim.item()))
    sims = sorted(sims, key=lambda x: x[1], reverse=True)
    for w, s in sims:
        print(w+"\t"+str(s))

## BERT

In [68]:
# bert_tokenizer = torch.hub.load('huggingface/pytorch-transformers', 'tokenizer', 'bert-base-cased')
# bert_model = torch.hub.load('huggingface/pytorch-transformers', 'model', 'bert-base-cased')

bert_tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-cased')
bert_model = transformers.BertModel.from_pretrained('bert-base-cased')

def w2BERT(words, tokenizer=bert_tokenizer, model=bert_model):
    # Tokenized input with special tokens around it (for BERT: [CLS] at the beginning and [SEP] at the end)
    indexed_tokens = tokenizer.encode(" ".join(words), add_special_tokens=False)
    for i, idx in enumerate(indexed_tokens):
        if idx == 100:
            print(words[i]+" is unknown")

    # Convert inputs to PyTorch tensors
    tokens_tensor = torch.tensor([indexed_tokens])

    with torch.no_grad():
        encoded_layers, _ = model(tokens_tensor)
        print(encoded_layers.shape)
    return encoded_layers[0]

def w2BERT2(words, tokenizer=bert_tokenizer, model=bert_model, index=1, pool=True):
    encodings = torch.Tensor(len(words), 768)
    for i, word in enumerate(words):
        indexed_tokens = tokenizer.encode(word, add_special_tokens=True)
        if 100 in indexed_tokens: print(word+" is unknown")
#         print(tokenizer.convert_ids_to_tokens(indexed_tokens))
        with torch.no_grad():
            tokens_tensor = torch.tensor([indexed_tokens])
            encoded_layers, _ = model(tokens_tensor)
        if pool:
            encodings[i,:] = encoded_layers[0].mean(axis=0)
        else:
            encodings[i,:] = encoded_layers[0,index]
#     encodings = encodings / encodings.norm(dim=1)[:, None]
    return encodings

print(bert_tokenizer.all_special_ids)
print(bert_tokenizer.all_special_tokens)

[101, 0, 103, 100, 102]
['[CLS]', '[PAD]', '[MASK]', '[UNK]', '[SEP]']


In [71]:
words = ["man", "woman", "car", "tractor", "bee", "steak", "vehicle", "motor", "potato", "carrot", "food", "farmer", "fish", "water", "metal", "she", "he", "programmer", "computer"]
encodings = w2BERT2(words, index=0, pool=True)
print(encodings.shape)

similarity(words, encodings, 0)

torch.Size([19, 768])
Similarity with 'man'
man	1.0
woman	0.9406140446662903
he	0.9063779711723328
she	0.902837336063385
farmer	0.8989346027374268
computer	0.8856185078620911
metal	0.8848690986633301
vehicle	0.8794581294059753
car	0.8697401881217957
bee	0.861171543598175
tractor	0.8602543473243713
programmer	0.8572993874549866
steak	0.8522169589996338
motor	0.8498015403747559
potato	0.8467987179756165
food	0.8389607667922974
fish	0.8364245891571045
water	0.8343541026115417
carrot	0.8342726826667786


## XLNet

In [104]:
xlnet_tokenizer = transformers.XLNetTokenizer.from_pretrained('xlnet-base-cased')
xlnet_model = transformers.XLNetModel.from_pretrained('xlnet-base-cased')

def w2xlnet(words, tokenizer=xlnet_tokenizer, model=xlnet_model):
    indexed_tokens = tokenizer.encode("<sep>".join(words), add_special_tokens=True)[:-1] # Exclude final <cls>
    for i, idx in enumerate(indexed_tokens):
        if idx == 0:
            print(words[i]+" is unknown")
    print(tokenizer.convert_ids_to_tokens(indexed_tokens))

    # Convert inputs to PyTorch tensors
    tokens_tensor = torch.tensor([indexed_tokens])

    with torch.no_grad():
        encoded_layers = model(tokens_tensor)
    return encoded_layers[0]

def w2xlnet2(words, tokenizer=xlnet_tokenizer, model=xlnet_model, index=0, pool=True):
    encodings = torch.Tensor(len(words), 768)
    for i, word in enumerate(words):
        indexed_tokens = tokenizer.encode(word, add_special_tokens=True)
        if 0 in indexed_tokens: print(word+" is unknown")
#         print(tokenizer.convert_ids_to_tokens(indexed_tokens))
        with torch.no_grad():
            tokens_tensor = torch.tensor([indexed_tokens])
            encoded_layers, = model(tokens_tensor)
        if pool:
            encodings[i,:] = encoded_layers[0].mean(axis=0)
        else:
            encodings[i,:] = encoded_layers[0,index]
#     encodings = encodings / encodings.norm(dim=1)[:, None]
    return encodings

print(xlnet_tokenizer.all_special_ids)
print(xlnet_tokenizer.all_special_tokens)

[3, 2, 1, 4, 7, 6, 5, 0, 8]
['<cls>', '</s>', '<s>', '<sep>', '<eod>', '<mask>', '<pad>', '<unk>', '<eop>']


In [122]:
words = ["man", "woman", "car", "king", "queen", "tractor", "nerd", "bee", "steak", "vehicle", "motor", "potato", "carrot", "food", "farmer", "fish", "water", "metal", "she", "he", "programmer", "computer"]
encodings = w2xlnet2(words, index=1, pool=False)
print(encodings.shape)

similarity(words, encodings, 2)

torch.Size([22, 768])
Similarity with 'car'
car	1.0
tractor	0.9669960737228394
computer	0.964198112487793
queen	0.9582439661026001
water	0.9576461911201477
fish	0.957129955291748
king	0.9553025960922241
potato	0.9539452791213989
vehicle	0.9538944363594055
metal	0.9533132314682007
motor	0.9527180790901184
farmer	0.9509403109550476
steak	0.950559139251709
carrot	0.9503024816513062
programmer	0.9501314163208008
woman	0.9483902454376221
he	0.9483819603919983
food	0.9464730024337769
man	0.9384380578994751
she	0.9346848130226135
bee	0.920799732208252
nerd	0.901278555393219


## XLNet/BERT words to embeddings

In [3]:
# load google news word2vec
E = WordEmbedding('./embeddings/w2v_gnews_small.txt')
print(E)
words = E.words
print("Words:", len(words))

*** Reading data from ./embeddings/w2v_gnews_small.txt
(26423, 300)
26423 words of dimension 300 : in, for, that, is, ..., Jay, Leroy, Brad, Jermaine
Words: 26423


In [5]:
def save_w2XLNet(words, filename, index=0, pool=True, specials=True):
    xlnet_tokenizer = transformers.XLNetTokenizer.from_pretrained('xlnet-base-cased')
    xlnet_model = transformers.XLNetModel.from_pretrained('xlnet-base-cased')
    unknowns = []
    with open(filename, "w") as f:
        for i, word in enumerate(words):
            if i % 100 == 0:
                print(i)
            f.write(word+" ")
            indexed_tokens = xlnet_tokenizer.encode(word, add_special_tokens=specials)
            if 0 in indexed_tokens:
                print(word+" is unknown")
                unknowns.append(word)
    #       print(tokenizer.convert_ids_to_tokens(indexed_tokens))
            with torch.no_grad():
                tokens_tensor = torch.tensor([indexed_tokens])
                encoded_layers, = xlnet_model(tokens_tensor) # Tensor
                if pool:
                    encoded_layers = encoded_layers[0].mean(axis=0)
                else:
                    encoded_layers = encoded_layers[0, index]
                vector_list = encoded_layers.tolist()
                vector_list = [str(x) for x in vector_list]
                f.write(" ".join(vector_list)+"\n")

save_w2XLNet(words, "xlnet_specials_meanpool.csv")

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18

In [4]:
def save_w2Bert(words, filename, index=1, pool=True, specials=True):
    bert_tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-cased')
    bert_model = transformers.BertModel.from_pretrained('bert-base-cased')
    unknowns = []
    with open(filename, "w") as f:
        for i, word in enumerate(words):
            if i % 100 == 0:
                print(i)
            f.write(word+" ")
            indexed_tokens = bert_tokenizer.encode(word, add_special_tokens=specials)
            if 0 in indexed_tokens:
                print(word+" is unknown")
                unknowns.append(word)
    #       print(tokenizer.convert_ids_to_tokens(indexed_tokens))
            with torch.no_grad():
                tokens_tensor = torch.tensor([indexed_tokens])
                encoded_layers, _ = bert_model(tokens_tensor)
                if pool:
                    encoded_layers = encoded_layers[0].mean(axis=0)
                else:
                    encoded_layers = encoded_layers[0, index]
                vector_list = encoded_layers.tolist()
                vector_list = [str(x) for x in vector_list]
                f.write(" ".join(vector_list)+"\n")

save_w2Bert(words, "bert_specials_meanpool.csv", pool=True, specials=True)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18